<a href="https://colab.research.google.com/github/maraprik/TMLC_GenAI/blob/main/Finetuning_week2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    LlamaTokenizer, BitsAndBytesConfig
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:

# The instruction dataset to use
dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

# Fine-tuned model name
new_model = "tinyllama-2-1B-kripa"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 4

# Alpha parameter for LoRA scaling
lora_alpha = 8

# Dropout probability for LoRA layers
lora_dropout = 0.2

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.float16

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=use_nested_quant,
)


In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Set the padding token to the end-of-sequence (eos) token
# This ensures compatibility when the model processes inputs with padding
tokenizer.pad_token = tokenizer.eos_token
# Configure the tokenizer to apply padding on the right side of the input
# This is often the default for causal language models to ensure alignment during training or inference
tokenizer.padding_side = "right"

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
# Defining the instruction that will guide the assistant's behavior for providing customer support answers.

instruction = """You are a helpful and efficient customer support bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.

Always prioritize accuracy and clarity in your answers.
If the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.
Provide useful links or references to detailed documentation when appropriate.
Use a friendly and professional tone, ensuring the response is easy to understand.
If the FAQ does not cover the question, offer an apology and suggest contacting customer support.
"""

def template(row):
    # Creating a list of message exchanges (system, user, assistant)
    row_json = [{"role": "system", "content": instruction }, # System message with the pre-defined instructions
               {"role": "user", "content": row["instruction"]}, # User's question from the dataset
               {"role": "assistant", "content": row["response"]}] # The assistant's answer from the dataset

    # Tokenizing the chat template and storing the result in the 'text' column (without applying tokenization)
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Applying the template function to each row in the dataset using multi-processing (4 processes in parallel)
dataset = dataset.map(template,num_proc= 4)

Map (num_proc=4):   0%|          | 0/21497 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5375 [00:00<?, ? examples/s]

In [ ]:
# Configure LoRA (Low-Rank Adaptation) for fine-tuning the model on a language modeling task
lora_peft_config = LoraConfig(
    lora_alpha=lora_alpha,        # Scaling factor
    lora_dropout=lora_dropout,    # Regularization dropout
    r=lora_r,                     # Rank for low-rank matrices
    bias="none",
    task_type="CAUSAL_LM",        # For language modeling
)


In [ ]:
# Setting up training arguments for the model training process
training_arguments = TrainingArguments(
    output_dir=output_dir,  # Directory where the results will be saved
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=5,  # Number of warmup steps to gradually increase the learning rate during training
    learning_rate=2e-4,
    fp16=True,  # Enabling 16-bit floating point precision for faster training on GPUs that support it (reduces memory usage)
    report_to="none",  # Disabling logging/reporting to external services (e.g., TensorBoard, Weights & Biases)
    #num_train_epochs=num_train_epochs, # Number of training epochs
    #per_device_train_batch_size=per_device_train_batch_size,
    #gradient_accumulation_steps=gradient_accumulation_steps,
    #optim=optim,
    #save_steps=save_steps,
    #logging_steps=logging_steps,
    #learning_rate=learning_rate, # Learning rate for the optimizer
    #weight_decay=weight_decay,
    #fp16=fp16,
    bf16=bf16,
    #max_grad_norm=max_grad_norm,
    #max_steps=max_steps,
    #warmup_ratio=warmup_ratio,
    #group_by_length=group_by_length,
    #lr_scheduler_type=lr_scheduler_type,
)

# Initializing the SFTTrainer for supervised fine-tuning
trainer = SFTTrainer(
    model=model,  # The pre-trained model to be fine-tuned
    train_dataset=dataset["train"], # The dataset used for training
    eval_dataset=dataset["test"],  # The dataset used for validation
    tokenizer=tokenizer,  # Tokenizer to process input text for the model
    args=training_arguments,  # The training arguments defined above
    peft_config=lora_peft_config,
)

<ipython-input-18-1528cd77d533>:29: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/21497 [00:00<?, ? examples/s]

Map:   0%|          | 0/5375 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss
500,0.605300
1000,0.475200
1500,0.433500
2000,0.422200
2500,0.409800
3000,0.393700
3500,0.388600
4000,0.379900
4500,0.377500
5000,0.372900
